# PureGym – Customer Review Topic & Emotion Analysis (Google + Trustpilot)

## Business problem
PureGym receives a high volume of customer reviews across platforms. Manually reading reviews does not scale, and it makes it difficult to identify recurring operational issues by location.

## Objective
Build a repeatable NLP pipeline that:
- Cleans and standardises review text across sources
- Surfaces recurring issues via topic modelling (BERTopic; benchmark with LDA)
- Adds an emotion signal to better prioritise high-frustration feedback
- Produces location-level insights to support operational prioritisation

## Data
Two sources of English-language reviews:
- Google reviews (rating, location, comment text)
- Trustpilot reviews (stars, location, review content)

## Scope and assumptions
- Ratings < 3 are treated as negative reviews for both sources.
- Location names are used as the join key; minor naming inconsistencies may reduce matches.
- Topic models provide directional insights and require human validation before action.


#1. Large language Model from Hugging Face

##1.1 Importing packages and data





In [ ]:
!pip install -U -q plotly==5.19.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 59.5 MB/s eta 0:00:00


In [ ]:
import time
# Start time to track the total execution time of the notebook
start_time = time.time()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
file_path = "/content/drive/MyDrive/Colab Notebooks/Course 3/Project/all_bad_reviews.pkl"
print("File size:", os.path.getsize(file_path), "bytes")

File size: 2269337 bytes


In [ ]:
#load "all_bad_reviews.pkl" saved file from google drive
import pickle

with open("/content/drive/MyDrive/Colab Notebooks/Course 3/Project/all_bad_reviews.pkl","rb") as f:
  data=pickle.load(f)

all_bad_reviews = data["all_bad_reviews"]
all_bad_reviews

,Review Content,Review Stars,Location Name,processed_comments
0,"Extremely busy, no fresh air.",1,Sutton Times Square,extremely busy fresh air
1,The men’s changing rooms smell bad. They need ...,2,Leeds City Centre South,men changing room smell bad deep clean sort sm...
2,No one was cleaning the equipment after use. C...,1,Dunfermline,cleaning equipment cleaning station hidden awa...
3,Not the best experience at 7am on a week day. ...,1,Bristol Harbourside,best experience bought pas received receipt ac...
4,Staff have their hands tied but surely head of...,1,Sunderland,staff hand tied surely head office bring rule ...
...,...,...,...,...
3942,"Equipment 7/10\n\nThey're pretty good, over th...",2,London Leytonstone,equipment pretty year attachment added variety...
3943,"""Alright for the price. Some members are a bit...",2,London Bermondsey,alright price member bit loud probably include...
3944,"Disorganised, cramped, and untidy. The old fre...",1,Bristol Union Gate,disorganised cramped untidy old free weight ar...
3945,"music always too loud, especially when cycling...",1,Bristol Union Gate,music loud especially cycling studio class com...


In [ ]:
#load "all_bad_reviews_list_clean.pkl" saved file from google drive
import pickle

with open("/content/drive/MyDrive/Colab Notebooks/Course 3/Project/all_bad_reviews_list_clean.pkl","rb") as f:
  data=pickle.load(f)

all_bad_reviews_list_clean = data["all_bad_reviews_list_clean"]


## 1.2 Initialize LLM

In [ ]:
!pip install -U transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 90.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
import transformers
print(transformers.__version__)

4.56.2


In [ ]:
model_id = "microsoft/Phi-4-mini-instruct"

# Load tokenizer & model into Colab local cache (default path: /root/.cache/huggingface)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"  # uses GPU if available, else CPU
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


In [ ]:
len(all_bad_reviews_list_clean)

3947

In [ ]:
generation_args_1 = {
    "max_new_tokens": 500, #500,
    "return_full_text": False,
    "do_sample": False,
}

In [ ]:
from tqdm import tqdm

topics_llm = []
#print(len(all_bad_reviews_list_clean))
iter = 0
for review in tqdm(all_bad_reviews_list_clean[0:100]): # subset for speed
  if len(str(review)) < 1000:   # Taking 1000 to skip very long reviews
    messages_2 = [
        {"role": "system", "content": "You work as a data analyst insights guru for a  large gym company in the UK and you want to find topics for improvments from customer reviews. You should return these in an array of strings only ['topic 1', 'topic 2', 'topic 3', ...]"},
        {"role": "user", "content": "In the following customer review interaction pick out a maximum of 3 main topics and return them as an array of topics: The showers are disgusting"},
        {"role": "assistant", "content": "['Shower cleanliness', 'odour', 'customer discomfort']"},
        {"role": "user", "content": f"In the following customer review interaction pick out the main 3 topics and return them as an array of topics: {review}"},
      ]

    print(review)
    # Flatten into string
    #chat_prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages_2]) + "\nAssistant:"
    #print(chat_prompt)
    output = pipe(messages_2, **generation_args_1)
    print("Output:" + str(output))


    #print(sequences[0]['generated_text'])
    topic_list_string = output[0]['generated_text'].replace("'", '"')
    topic_list = json.loads(topic_list_string)

    topics_llm.append(topic_list)



  0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


extremely busy fresh air


  1%|          | 1/100 [00:08<13:46,  8.34s/it]

Output:[{'generated_text': "['Crowded facilities', 'air quality', 'customer experience']"}]
men changing room smell bad deep clean sort smell management toilet pleasant put using facility regular cleaning enough top


  2%|▏         | 2/100 [00:16<13:18,  8.15s/it]

...

Output:[{'generated_text': "['Customer service experience', 'payment and refund process', 'accessibility and convenience']"}]
look cleaner site toilet clean smelly water flooded floor sink terrible experience


 99%|█████████▉| 99/100 [12:21<00:08,  8.81s/it]

Output:[{'generated_text': "['Site cleanliness', 'toilet condition', 'flooding','sink condition', 'overall experience']"}]
went open eve greet show around etc promotion video ensure great experience extremely busy pretty putting comfortable walked round asked inwhich reply never normally staff reply open eve event reply right completely clueless intimating best time experience welcome new member need staffing holding event


100%|██████████| 100/100 [12:33<00:00,  7.53s/it]

Output:[{'generated_text': "['Staff responsiveness', 'Event organization', 'Customer experience', 'Staff training', 'Event timing', 'Event capacity']"}]


##1.3 BERTopic

In [ ]:
!pip install bertopic -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 4.6 MB/s eta 0:00:00


In [ ]:
from bertopic import BERTopic

# Flatten your nested list of topics
topic_string_array=[topic for sublist in topics_llm for topic in sublist]

# Fit BERTopic
topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(topic_string_array)

print("topic_string_array:", topic_string_array)
# Inspect results
print(topic_model.get_topic_info().head())

/usr/local/lib/python3.12/dist-packages/hdbscan/plots.py:448: SyntaxWarning: invalid escape sequence '\l'
  axis.set_ylabel('$\lambda$ value')
/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

topic_string_array: ['Crowded facilities', 'air quality', 'customer experience', 'Men changing room cleanliness', 'smell management', 'toilet cleanliness', 'Equipment cleanliness', 'Accessibility of cleaning station', 'Staff presence and assistance', 'Safety concerns', 'Delivery process', 'Payment issues', 'Customer service', 'Staff behavior', 'equipment availability', 'gym atmosphere', 'Support accessibility', 'Manager responsiveness', 'Code access', 'Staff behavior', 'Customer experience', 'Facility cleanliness and maintenance', 'Ventilation', 'Air quality', 'Disinfectant use', 'Blocked access', 'Manager issues', 'Account problems', 'Air conditioning unit performance', 'temperature control', 'customer service responsiveness', 'Cancellation process', 'Induction experience', 'Car park management', 'Value for money', 'Membership joining process', 'Payment issues', 'Customer service experience', 'Staff behaviour', 'Work environment', 'Customer experience', 'Equipment availability', 'Work

In [ ]:
topic_model.get_topic_info().head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,13,-1_service_age_quality_policy,"[service, age, quality, policy, crowded, hones...","[service quality, Service quality, Service qua..."
1,0,51,0_staff_manager_training_responsiveness,"[staff, manager, training, responsiveness, beh...","[Staff responsiveness, Staff responsiveness, S..."
2,1,46,1_process_payment_access_membership,"[process, payment, access, membership, issues,...","[Membership access, Membership joining process..."
3,2,39,2_cleanliness_facility_cleaning_smell,"[cleanliness, facility, cleaning, smell, toile...","[Facility cleanliness, Facility cleanliness, F..."
4,3,29,3_concerns_safety_dissatisfaction_issues,"[concerns, safety, dissatisfaction, issues, co...","[Safety concerns, Safety concerns, Customer sa..."


In [ ]:
topic_model.get_topic(-1)

[('service', np.float64(0.25713531062751366)),
 ('age', np.float64(0.2429426701187254)),
 ('quality', np.float64(0.22481338191844447)),
 ('policy', np.float64(0.18097834385410227)),
 ('crowded', np.float64(0.1456263372823471)),
 ('honesty', np.float64(0.1456263372823471)),
 ('expectation', np.float64(0.1456263372823471)),
 ('enforcement', np.float64(0.1456263372823471)),
 ('restrictions', np.float64(0.1456263372823471)),
 ('understanding', np.float64(0.1456263372823471))]

In [ ]:
#topic_model.get_topic(1)

##1.4 Get Actionable Insight from LLM

In [ ]:
messages_3 = [
    {"role": "system", "content": "You work as a data analyst insights guru for a large gym company in the UK and you want to compress a given list into collated topics. You should return these in a numbered list as business insights that can be used to improve the business"},
    {"role": "user", "content": "In the following list containing the main extracted topics from customer reviews, group or compress the topics and return them with actionable insights in a numbered list: {topic_string_array}"},
    {"role": "assistant", "content": "1. Customer Support: Improve response times and staff availability at reception and helplines to enhance member satisfaction. \n 2. Facilities & Maintenance: Implement a proactive equipment maintenance schedule and regular cleanliness checks.\n ..."},
    {"role": "user", "content": f"In the following list containing the main extracted topics from customer reviews, group or compress the topics and return them with actionable insights in a numbered list: {topic_string_array}"},
]



In [ ]:
output = pipe(messages_3, **generation_args_1)

#print(sequences[0]['generated_text'])
insights = output[0]['generated_text']

In [ ]:
insights

'1. Facility Cleanliness and Maintenance: Implement a proactive cleaning schedule, improve ventilation, and ensure regular maintenance of equipment and facilities to enhance customer satisfaction and safety.\n2. Customer Service and Staff Behavior: Improve staff responsiveness, provide better training, and ensure a friendly and professional attitude to enhance customer experience and satisfaction.\n3. Equipment Availability and Maintenance: Regularly maintain and clean equipment, ensure availability, and address any issues promptly to improve customer satisfaction and safety.\n4. Customer Experience and Satisfaction: Improve the overall customer experience by addressing issues related to cleanliness, staff behavior, and service quality, and ensure prompt resolution of customer concerns.\n5. Payment and Billing Issues: Streamline the payment process, address billing issues promptly, and ensure clear communication regarding membership fees and cancellation policies.\n6. Website Usability

In [ ]:
print(insights)

1. Facility Cleanliness and Maintenance: Implement a proactive cleaning schedule, improve ventilation, and ensure regular maintenance of equipment and facilities to enhance customer satisfaction and safety.
2. Customer Service and Staff Behavior: Improve staff responsiveness, provide better training, and ensure a friendly and professional attitude to enhance customer experience and satisfaction.
3. Equipment Availability and Maintenance: Regularly maintain and clean equipment, ensure availability, and address any issues promptly to improve customer satisfaction and safety.
4. Customer Experience and Satisfaction: Improve the overall customer experience by addressing issues related to cleanliness, staff behavior, and service quality, and ensure prompt resolution of customer concerns.
5. Payment and Billing Issues: Streamline the payment process, address billing issues promptly, and ensure clear communication regarding membership fees and cancellation policies.
6. Website Usability and A

This LLM output reframes the granular complaint clusters from previous Runs 1–3 into strategic improvement themes.

Core issues are consistent (cleanliness, equipment, staff, payments, parking, atmosphere), but the LLM elevates them into broader categories such as Customer Safety and Security and Website Usability.

This abstraction introduces some redundancy (e.g. multiple customer service/satisfaction categories), but also surfaces new actionable angles:

- Safety as a distinct management focus (lockers, overcrowding, fire alarms).

- Website usability as part of the access/joining/cancellation journey.

- Communication as an organisational capability, not just staff politeness.


Compared with BERTopic’s raw clusters, this output is less diagnostic but more strategic — suitable for senior stakeholders and action planning.

In [ ]:
# End time to track the total execution time of the notebook
time.time()
end_time = time.time()
total_time = end_time - start_time
# Convert to hours, minutes, and seconds
mins, secs = divmod(total_time, 60)
hrs, mins = divmod(mins, 60)

print(f"Total execution time: {int(hrs)}h {int(mins)}m {int(secs)}s")

Total execution time: 0h 22m 34s
